## Top Novels

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import re
import jieba
import jieba.posseg as psg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [2]:
results={'URL':[], 'Class':[]}

In [25]:
url_collect=['https://www.jjwxc.net/bookbase.php?fw0=0&fbsj0=0&ycx0=0&xx1=1&mainview0=0&sd1=1&lx0=0&fg0=0&bq=&removebq=&sortType=0&isfinish=0&collectiontypes=ors&searchkeywords=&page=500','https://www.jjwxc.net/bookbase.php?fw0=0&fbsj0=0&ycx0=0&xx1=1&mainview0=0&sd1=1&lx0=0&fg0=0&bq=&removebq=&sortType=0&isfinish=0&collectiontypes=ors&searchkeywords=&page=600','https://www.jjwxc.net/bookbase.php?fw0=0&fbsj0=0&ycx0=0&xx1=1&mainview0=0&sd1=1&lx0=0&fg0=0&bq=&removebq=&sortType=0&isfinish=0&collectiontypes=ors&searchkeywords=&page=800']

In [28]:
for i in url_collect:
    content=requests.get(i)
    soup=BeautifulSoup(content.content,'lxml')
    tbody=soup.find('tbody')
    print(tbody)
    novel_all=tbody.find_all('tr')
    for n in novel_all[1:]:
        url=n.find('a',{'class':'tooltip'}).get('href')
        cla=n.find('td',{'align':'center'},{'height':'23'}).next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text.strip()
        #points=n.find('td',{'align':'right'},{'bgcolor':'#eefaee'}).next_sibling.next_sibling.text.strip()
        results['URL'].append(url)
        #results['Points'].append(points)
        results['Class'].append(cla)

<tbody>
<tr>
<td class="sptd" width="126">作者</td>
<td class="sptd" width="278">作品</td>
<td class="sptd" width="168">类型</td>
<td class="sptd" width="71">风格</td>
<td class="sptd" width="63">进度</td>
<td class="sptd" width="63">字数</td>
<td class="sptd" width="73">作品积分</td>
<td class="sptd" width="138">发表时间</td>
</tr>
<tr>
<td align="left">
<a href="oneauthor.php?authorid=2380570" target="_blank">入云深处皆沾衣</a>
</td>
<td align="left">
<a href="onebook.php?novelid=6892844" target="_blank" title="简介：利用尾田画不死人的特点活下去
标签：海贼王 穿越时空 少年漫 大冒险">我在穷凶极恶的海贼世界抱紧论坛</a>
</td>
<td align="center">
                                    衍生-言情-近代现代-其他衍生                                </td>
<td align="center">
                                    正剧                                </td>
<td align="center">
                                                                        连载中                                </td>
<td align="right">100467</td>
<td align="right">2537663</td>
<td align="center">2022-05-18 21:52:28</td>


AttributeError: 'NoneType' object has no attribute 'get'

In [5]:
results_df=pd.DataFrame.from_dict(results)

In [7]:
results_df=results_df.drop_duplicates()
results_df=results_df[results_df['Class'] == '原创-言情-近代现代-爱情']
add_str='https://www.jjwxc.net/'
results_df["URL"]=add_str+results_df["URL"]
#results_df['Points']=results_df['Points'].str.replace(',', '').astype(int)

In [8]:
results_df

,URL,Class
0,https://www.jjwxc.net/onebook.php?novelid=3676002,原创-言情-近代现代-爱情
1,https://www.jjwxc.net/onebook.php?novelid=4001734,原创-言情-近代现代-爱情
2,https://www.jjwxc.net/onebook.php?novelid=3525826,原创-言情-近代现代-爱情
3,https://www.jjwxc.net/onebook.php?novelid=3694555,原创-言情-近代现代-爱情
4,https://www.jjwxc.net/onebook.php?novelid=4836678,原创-言情-近代现代-爱情
...,...,...
1180,https://www.jjwxc.net/onebook.php?novelid=4926749,原创-言情-近代现代-爱情
1182,https://www.jjwxc.net/onebook.php?novelid=3459704,原创-言情-近代现代-爱情
1190,https://www.jjwxc.net/onebook.php?novelid=2713183,原创-言情-近代现代-爱情
1194,https://www.jjwxc.net/onebook.php?novelid=1461220,原创-言情-近代现代-爱情


In [17]:
novel_info={'Name':[],'Style':[], 'Finish':[], 'Words':[],'Intro':[],'Short_Intro':[], 'Points':[]}

In [18]:
for u in results_df['URL']:
    novel_con=requests.get(u)
    soup_con=BeautifulSoup(novel_con.content,'lxml')
    name=soup_con.h1.text
    intro=soup_con.find('div', {'id' : 'novelintro'}).text
    intro_short=soup_con.find('span', {'style' : 'color:#F98C4D'}).text[6:]
    basic=soup_con.find('ul', {'class' : 'rightul'})
    style=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.text[5:]
    finish=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text[6:]
    word=basic.li.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text[6:][:-2]
    points=soup_con.find('tbody').find('span',{'itemprop':'collectedCount'}).next_sibling.next_sibling.next_sibling.strip()[5:]

    novel_info['Name'].append(name)
    novel_info['Style'].append(style)
    novel_info['Finish'].append(finish)
    try:
        novel_info['Words'].append(word)
    except:
        novel_info['Words'].append(None)
    novel_info['Intro'].append(intro)
    novel_info['Short_Intro'].append(intro_short)
    novel_info['Points'].append(points)

In [19]:
novel_df=pd.DataFrame.from_dict(novel_info)
novel_df['Points']=novel_df['Points'].str.replace(',', '').astype(int)
novel_df

,Name,Style,Finish,Words,Intro,Short_Intro,Points
0,偷偷藏不住,轻松,完结\n,372127,※实体书已出版上市，各平台均有售=v=十三岁那年，桑稚偷偷喜欢上一个男人。男人的模样冷淡慵懒...,少女暗恋成真,16849676288
1,难哄,轻松,完结\n,389888,【全文完，下本写《折月亮》】机缘巧合之下，温以凡跟曾被她拒绝过的高中同学桑延过上了合租的生活...,合租室友总以为我想泡他,16409806848
2,白日梦我,正剧,完结\n,427269,【下本《桃枝气泡》】【实体书已出版，淘宝天猫当当等平台均有销售ww微博@栖见吗】奖学金争夺战...,倦爷今天也真实又努力地穷着,9628956672
3,偏偏宠爱,轻松,完结\n,336762,【出版名《偏偏宠爱》，收录新番外“仙女的青睐”等，当当、淘宝、京东均可购买。】【本文已完结，...,被神经病少年追求的那些年,8515629056
4,告白,正剧,完结\n,438948,【隔壁新文《你听得见》已开】【《告白》上册最后一批签名纸已寄回印厂，感谢等待。2022.6....,假以时日美梦成真,9793348608
...,...,...,...,...,...,...,...
567,裙下臣,正剧,完结\n,456978,【下本开《无声的世界，还有他》】【预收文《不知如何爱你时》，专栏可预收】----------...,周年纪念日要过三次的女人,3350039296
568,我是大反派[快穿],轻松,完结\n,1742010,隔壁同类型新文《快叫我爸爸（快穿）》正在火热更新中人渣or反派的漫漫改造之路暂定世界：抛弃妻...,大反派or渣男改造之旅,3333573632
569,快穿之教你做人,轻松,完结\n,1733164,【穿越各个世界，教极品们做人。】备注：1.苏爽文。2.男主始终是一个人，没切片，没失忆，没人...,苏爽快穿,3262351872
570,大战拖延症,轻松,完结\n,177656,资深拖延症患者叶子璐，一直在这个“忠实的好朋友”的纠缠、折磨和慰藉下凑合活着。直到床头的旧布...,打倒拖延症,3237918720


In [22]:
novel_df.to_csv('jjwxc1.csv')